## <div style="color:white;display:fill;border-radius:5px;background-color:#264653;letter-spacing:0.5px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;text-align: left;margin:0;font-size:120%">9.0 Deploy Model to Production</p></div>

In [ ]:
# Salvar modelo treinado
#pickle.dump( xgb_model_tunned, open('../src/models/model_health_insurance.pkl', 'wb'))

#### <div style="color:white;display:fill;border-radius:5px;background-color:#f4a261;letter-spacing:0.5px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;text-align: left;margin:0;font-size:80%"> HealthInsurance Class</p></div> 

In [55]:
import pickle
import numpy as np
import pandas as pd
import inflection

class HealthInsurance:
    
    def __init__(self):
        self.home_path              = 'C:\\Users\\yago2\\OneDrive\\Documentos\\Repus\\healt_insurance_cross_sell_project\\'
        self.annual_premium_scaler  = pickle.load( open ( self.home_path + 'src/features/annual_premium_scaler.pkl', 'rb'))
        self.age_scaler             = pickle.load( open ( self.home_path + 'src/features/age_scaler.pkl', 'rb' ))
        self.vintage_scaler         = pickle.load( open ( self.home_path + 'src/features/vintage_scaler.pkl', 'rb' ))
        self.encode_region_code     = pickle.load( open ( self.home_path + 'src/features/encode_region_code.pkl', 'rb' ))
        self.encode_policy          = pickle.load( open ( self.home_path + 'src/features/encode_policy.pkl', 'rb' ))


    def rename_columns( df_train ):
        cols_old = df_train.columns

        cols_new = []
        cols_new = cols_old.map(lambda x: inflection.underscore(x))

        df_train.columns = cols_new
        
        return df_train

    
    def feature_engineering( self, df1 ):
        # vehicle_age
        df1['vehicle_age'] = df1['vehicle_age'].apply( lambda x: 'over_2_years' if x == '> 2 Years' else 'between_1_2_year' if x == '1-2 Year' else 'below_1_year')

        # vehicle_damage
        df1['vehicle_damage'] = df1['vehicle_damage'].apply( lambda x: 1 if x == 'Yes' else 0 )
        
        return df1
    
    def data_encoding( self, df3 ):
        # gender - Label Encoder
        gender_dict = {'Male': 0, 'Female': 1}
        df3['gender'] = df3['gender'].map( gender_dict )

        #region_code - Frequency Encoding 
        df3.loc[ : , 'region_code'] = df3['region_code'].map( self.encode_region_code )
        
        #vehicle_age - Ordinal Encoding  
        vehicle_age_dict = {'below_1_year': 1, 'between_1_2_year': 2, 'over_2_years': 3}
        df3['vehicle_age'] = df3['vehicle_age'].map( vehicle_age_dict )

        # policy_sales_channel  *Frequency Encoding 
        df3.loc[ : , 'policy_sales_channel'] = df3['policy_sales_channel'].map( self.encode_policy )

        return df3
    
    def data_rescaling( self, df3 ):
        #annual_premium
        df3['annual_premium'] = self.annual_premium_scaler.transform( df3[['annual_premium']].values )

        #age
        df3['age'] = self.age_scaler.transform( df3[['age']].values )
        
        #vintage
        df3['vintage'] = self.vintage_scaler.transform( df3[['vintage']].values )

        cols_selected = [ 'age', 'region_code', 'previously_insured', 'vehicle_damage', 'annual_premium','policy_sales_channel', 'vintage']

        return df3[ cols_selected ]
    
    def get_prediction( self, model, original_data, test_data):
        #model prediction
        pred = model.predict_proba( test_data)
        #join prediction into original data
        original_data['score'] = pred

        return original_data.to_json( orient='records' , date_format='iso')
        
        

#### <div style="color:white;display:fill;border-radius:5px;background-color:#f4a261;letter-spacing:0.5px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;text-align: left;margin:0;font-size:70%">API Handler</p></div> 

In [ ]:
import pickle
import pandas as pd
import os
from flask import Flask, request, Response
from healthinsurance.HealthInsurance import HealthInsurance

# loading model
path = 'C:\\Users\\yago2\\OneDrive\\Documentos\\Repus\\healt_insurance_cross_sell_project\\'
model = pickle.load( open( path + 'src/models/model_health_insurance.pkl', 'rb') )

# initialize API
app = Flask( __name__ )

@app.route( '/healthinsurance/predict', methods=['POST'] )

def health_insurance_predict():
    test_json = request.get_json()

    if test_json: # there is data

        if isinstance( test_json, dict ): # unique example
            test_raw = pd.DataFrame( test_json, index=[0] )
        else: # multiple example
            test_raw = pd.DataFrame( test_json, columns=test_json[0].keys() )
        # Instantiate Healt Insurance class
        pipeline = HealthInsurance()
        # data cleaning
        df1 = pipeline.rename_columns( test_raw )
        # feature engineering
        df2 = pipeline.feature_engineering( df1 )
        # data encoding
        df3 = pipeline.data_encoding( df2 )
        # data rescaling
        df4 = pipeline.data_rescaling( df3 )
        # prediction
        df_response = pipeline.get_prediction( model, test_raw, df4 )

        return df_response
    
    else:
        return Response( '{}', status=200, mimetype='application/json' )
    
#if __name__ == '__main__':
#    app.run( '0.0.0.0', debug=True )

if __name__ == '__main__':
    port = os.environ.get('PORT', 5000)
    app.run( '0.0.0.0', port=port )

#### <div style="color:white;display:fill;border-radius:5px;background-color:#f4a261;letter-spacing:0.5px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;text-align: left;margin:0;font-size:70%">API Tester</p></div> 

In [1]:
import pandas as pd
import requests
import json

In [2]:
data_test = pd.read_csv('../dataset/test.csv')

In [3]:
data_test

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,381110,Male,25,1,11.0,1,< 1 Year,No,35786.0,152.0,53
1,381111,Male,40,1,28.0,0,1-2 Year,Yes,33762.0,7.0,111
2,381112,Male,47,1,28.0,0,1-2 Year,Yes,40050.0,124.0,199
3,381113,Male,24,1,27.0,1,< 1 Year,Yes,37356.0,152.0,187
4,381114,Male,27,1,28.0,1,< 1 Year,No,59097.0,152.0,297
...,...,...,...,...,...,...,...,...,...,...,...
127032,508142,Female,26,1,37.0,1,< 1 Year,No,30867.0,152.0,56
127033,508143,Female,38,1,28.0,0,1-2 Year,Yes,28700.0,122.0,165
127034,508144,Male,21,1,46.0,1,< 1 Year,No,29802.0,152.0,74
127035,508145,Male,71,1,28.0,1,1-2 Year,No,62875.0,26.0,265


In [10]:
df_test = data_test.sample(10)

In [11]:
#df_test.to_csv('pa004_health_insurance.csv', index=False)

In [9]:
# Convert Dataframe to json
data = json.dumps( df_test.to_dict(orient='records'))

In [10]:
data

'[{"id": 393050, "Gender": "Male", "Age": 42, "Driving_License": 1, "Region_Code": 28.0, "Previously_Insured": 0, "Vehicle_Age": "1-2 Year", "Vehicle_Damage": "Yes", "Annual_Premium": 45325.0, "Policy_Sales_Channel": 26.0, "Vintage": 158}, {"id": 431168, "Gender": "Female", "Age": 73, "Driving_License": 1, "Region_Code": 28.0, "Previously_Insured": 0, "Vehicle_Age": "> 2 Years", "Vehicle_Damage": "Yes", "Annual_Premium": 33562.0, "Policy_Sales_Channel": 26.0, "Vintage": 30}, {"id": 381975, "Gender": "Male", "Age": 24, "Driving_License": 1, "Region_Code": 29.0, "Previously_Insured": 1, "Vehicle_Age": "< 1 Year", "Vehicle_Damage": "No", "Annual_Premium": 23905.0, "Policy_Sales_Channel": 152.0, "Vintage": 65}, {"id": 410461, "Gender": "Male", "Age": 66, "Driving_License": 1, "Region_Code": 7.0, "Previously_Insured": 0, "Vehicle_Age": "1-2 Year", "Vehicle_Damage": "No", "Annual_Premium": 2630.0, "Policy_Sales_Channel": 7.0, "Vintage": 46}, {"id": 492811, "Gender": "Female", "Age": 29, "Dri

In [11]:
# API Call
#url =  'http://0.0.0.0:5000//predict'
header = {'Content-type': 'application/json' }
data = data

r = requests.post( url, data=data, headers=header )
print( 'Status Code {}'.format( r.status_code ) )

Status Code 200


In [12]:
d1 = pd.DataFrame( r.json(), columns=r.json()[0].keys() )
d1.sort_values( 'score',ascending=False)

,id,gender,age,driving_license,region_code,previously_insured,vehicle_age,vehicle_damage,annual_premium,policy_sales_channel,vintage,score
0,393050,0,0.338462,1,0.279631,0,2,1,0.855046,0.209638,0.512111,0.407501
8,434302,0,0.492308,1,0.010496,0,2,1,-1.619422,0.027735,0.100346,0.240166
1,431168,1,0.815385,1,0.279631,0,3,1,0.173300,0.209638,0.069204,0.189134
7,498778,1,0.015385,1,0.279631,0,1,1,0.663325,0.015901,0.467128,0.088831
3,410461,0,0.707692,1,0.008639,0,2,0,-1.619422,0.004211,0.124567,0.037363
2,381975,0,0.061538,1,0.029070,1,1,0,-0.386390,0.353472,0.190311,0.001361
4,492811,1,0.138462,1,0.010496,1,1,0,-0.635836,0.353472,0.629758,0.000262
5,385092,1,0.800000,1,0.279631,1,2,0,1.056098,0.209638,0.982699,0.000160
9,381282,1,0.061538,1,0.006717,1,1,0,-0.473615,0.353472,0.564014,0.000105
6,420155,1,0.030769,1,0.029070,1,1,0,-0.192524,0.353472,0.525952,0.000102
